# DVL-ODO as Embedded Process
Gregory Burgess

03/26/2021

---

### Goals

1. The goal of this python script is to feed in the raw PDO data and Glider flight data iteratively in order to re-create the dives from Kolumbo 2019 and test the navigation algorithms as embedded processes. Ideally, the end product of this script will be an animation of the dive in "real-time" that shows both the DR track as well as the DVL-ODO track over time.

2. The main challenge is making sure the python classes being used to hold the DVL and flight computer data are robust enough to handle the data in an online fashion.

3. The next step will be to transfer this to ROS and embed the navigation processes with the backseat driver as well as construct two interfaces (one for the actual glider, and one for the simulator). 

---
<a id='import-libraries'></a>
## Import Libraries

In [2]:
import datetime
import importlib
import numpy as np
import os
import pandas as pd
import rasterio as rio
import scipy
import scipy.signal
import seaborn as sns 
import struct
import sys
import utm
import unittest
from PIL import Image
from matplotlib import pyplot as plt 
from os import listdir
from os.path import isfile, join
from scipy.spatial.transform import Rotation as R
from scipy import interpolate

# add parent directory to the path for importing modules 
sys.path.insert(1, os.path.join(sys.path[0], '..'))
sys.path.append(os.path.join(sys.path[0], '../data'))

# objects for parsing raw DVL data 
import PathfinderDVL
import PathfinderEnsemble
import PathfinderTimeSeries

# objects for estimating ocean current velocities
import VelocityShearPropagation

# objects for controlling thruster to minimize transport cost 
import AdaptiveVelocityController

# objects for parsing flight and science computer log files
import SlocumFlightController
import SlocumScienceController
import dvl_plotter
import BathymetryMap
import MultiFactorTAN

# data for parsing seafloor bathymetry
import bathy_meta_data
sns.set()


import warnings
warnings.simplefilter('ignore')

def reload_modules():
    importlib.reload(PathfinderDVL)
    importlib.reload(PathfinderEnsemble)
    importlib.reload(PathfinderTimeSeries)
    importlib.reload(VelocityShearPropagation)
    importlib.reload(AdaptiveVelocityController)
    importlib.reload(SlocumFlightController)
    importlib.reload(SlocumScienceController)
    importlib.reload(dvl_plotter)
    importlib.reload(bathy_meta_data)
    importlib.reload(BathymetryMap)
    importlib.reload(MultiFactorTAN)
print('Done!')

Done!


## Read in Glider Data, DVL data, and Bathy Data

In [3]:
reload_modules()

################## Read in Glider Data ####################
directory = "C:/Users/grego/Dropbox/Kolumbo cruise 2019/zduguid/dbd-parsed/sentinel_2019-Nov/"
ts_flight_kolumbo_all = SlocumFlightController.SlocumFlightController.from_directory(directory, save=False, verbose=False)

################## Read in bathymetric data ####################
#Grid_res_num should always be 10 to represent original resolution of the bathymetry chart. Then, the Minimum spatial resolution will be 5m
#10m resolution
bathy_df = pd.read_csv('C:/Users/grego/Dropbox/Kolumbo cruise 2019/zduguid/bathy/sensitivity_tests/Kolumbo-1.csv')
grid_res_num = 10
# map_var_resolution = '10m'
#20m resolution
bathy_df_var = pd.read_csv('C:/Users/grego/Dropbox/Kolumbo cruise 2019/zduguid/bathy/sensitivity_tests/Kolumbo-2.csv')
map_var_resolution = '20m'

################## Read in DVL data #########################
glider = "sentinel"
filepath = "C:/Users/grego/Dropbox/Kolumbo cruise 2019/zduguid/pd0-raw/%s/" % (glider)
    #################################################
    # File ID Number ################################
    #################################################
filename2  = "sk211652.pd0" # DIVE 2
filename3  = "01820002.pd0" # DIVE 3
filename4  = "sk220034.pd0" # DIVE 4
filename5  = "01820008.pd0" # DIVE 5
filename7  = "01820013.pd0" # DIVE 7
filename9  = "sk222256.pd0" # DIVE 9
filename12 = "sk230350.pd0" # DIVE 12
filename14 = "sk261222.pd0" # DIVE 14
    #################################################
    # Parse Selected File IDs #######################
    #################################################
# ts2  = PathfinderTimeSeries.PathfinderTimeSeries.from_pd0(filepath+filename2,  save=False)
ts3  = PathfinderTimeSeries.PathfinderTimeSeries.from_pd0(filepath+filename3,  save=False)
ts4  = PathfinderTimeSeries.PathfinderTimeSeries.from_pd0(filepath+filename4,  save=False)
ts5  = PathfinderTimeSeries.PathfinderTimeSeries.from_pd0(filepath+filename5,  save=False)
ts7  = PathfinderTimeSeries.PathfinderTimeSeries.from_pd0(filepath+filename7,  save=False)
ts9  = PathfinderTimeSeries.PathfinderTimeSeries.from_pd0(filepath+filename9,  save=False)
# ts12 = PathfinderTimeSeries.PathfinderTimeSeries.from_pd0(filepath+filename12, save=False)
ts14 = PathfinderTimeSeries.PathfinderTimeSeries.from_pd0(filepath+filename14, save=False)
    #################################################
    # Frontiers (and Thesis) Naming Convention ###################
    #################################################
# tsb  = ts12 # (no bottom)
# tsc  = ts2 # (no bottom)
# tsh  = ts9 # (not included in Frontiers) (also doesn't look like too much bottom?)
# tsd  = ts3 #(crazy dive)
tsa  = ts14
tse  = ts4
tsf  = ts5
tsg  = ts7 # (not included in Frontiers)
    #################################################
    # JFR Naming Convention #########################
    #################################################
# tsa  = ts14
# tsb  = ts5
# tsc  = ts4 
# tsd  = ts3
# tse = ts7

>> Parsing folder of ASC Files
>> Finished Parsing!
________________________________________
- Parsing DVL File ---------------------
    input file: 01820002.pd0
    # ensembles:    200
    # ensembles:    400
    # ensembles:    600
    # ensembles:    800
    # ensembles:   1000
    # ensembles:   1200
    # ensembles:   1400
    # ensembles:   1600
    # ensembles:   1800
    # ensembles:   2000
    # ensembles:   2200
    # ensembles:   2400
- Parsing Complete ---------------------
    # ensembles:   2410
    parsing time:  7.385799
- Sensor Configuration -----------------
    600kHz System
    Convex Beam Pattern
    Sensor Config #1
    Attached
    Down Facing
    30E Beam Angle
    4 Beam Janus
- Coordinate Transformation ------------
    Bin Mapping Used
    3-Beam Soln Used
    Tilts Used
    Earth Coords
________________________________________
- Parsing DVL File ---------------------
    input file: sk220034.pd0
    # ensembles:    200
    # ensembles:    400
    # ensembl

## Select Dive to Simulate

In [4]:
#Select Time Series to Run Naviagation Code on
#ts_label is purely for visual results at end (not important for code)
ts = tsa
dive_label = 'A'

In [9]:
def pd0_read(fn):
    data = open(fn, 'rb').read()
    ensembles = 0
    while len(data) > 0:
        parsed = pd0_parser.parse_pd0_bytearray(data)
        print(parsed)
        # Chop off the ensemble we just parsed
        data = data[parsed['header']['number_of_bytes'] + 2:]
        ensembles += 1
    print(ensembles, 'ensembles')

desired_pd0_data = filename14
file = filepath + desired_pd0_data

#####Open pd0 Data ######
data = open(file, 'rb').read()      #opens file in read mode and binary mode



NameError: name 'pd0_parser' is not defined